In [ ]:
import os,pdb
from transformers import MaskFormerImageProcessor, MaskFormerForInstanceSegmentation, SegformerForSemanticSegmentation, SamModel, SamProcessor
from PIL import Image
from torchvision import transforms
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset


## Show Image

In [ ]:
# image = example['pixel_values']
# raw_image = Image.open('/scratch/alpine/skinder@xsede.org/glaucoma_segmentation_data/seg_public_all_vtwo/images/drishtiGS_054.png') # Scott
raw_image = Image.open('/sddata/data/MedSAM/public_test_data_01_10_2023/Color_Fundus/visivite_GA.png')
# Convert RGBA image to RGB
raw_image = raw_image.convert("RGB")

In [ ]:
np.array(raw_image)

In [ ]:
np.array(raw_image).shape

In [ ]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"
device = "cpu"

model = SamModel.from_pretrained("flaviagiammarino/medsam-vit-base").to(device)
processor = SamProcessor.from_pretrained("flaviagiammarino/medsam-vit-base")

# img_url = "https://huggingface.co/flaviagiammarino/medsam-vit-base/resolve/main/scripts/input.png"
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")


In [ ]:
processor

In [ ]:
model.config
# model.state_dict()

In [ ]:
raw_image_np = np.array(raw_image)
raw_image_np.shape # (1811, 2039, 3)
# Define transformations (resize, convert to tensor)
transform = transforms.Compose([
    transforms.Resize((1024, 1024)),  # Resize the image and mask to the desired input size
    # transforms.Resize((512, 512)),  # Resize the image and mask to the desired input size
    transforms.ToTensor(),           # Convert PIL Image to tensor
])

t_raw_image_np = transform(raw_image)
t_raw_image_np = t_raw_image_np.permute(1, 2, 0)
print(np.array(raw_image).shape)
print(t_raw_image_np.shape)



# Inference

In [ ]:
input_boxes = [600., 600., 1200., 1400.] # [x1, y1, x2, y2] (1811, 2039, 3)
# input_boxes = [300., 400., 600., 800.] # [x1, y1, x2, y2] torch.Size([1024, 1024, 3])
# input_boxes = [0., 0., 512., 512.]
inputs = processor(raw_image, input_boxes=[[input_boxes]], return_tensors="pt").to(device)
# inputs = processor(t_raw_image_np, input_boxes=[[input_boxes]], return_tensors="pt").to(device)
inputs
outputs = model(**inputs, multimask_output=False)
# outputs
probs = processor.image_processor.post_process_masks(
    outputs.pred_masks.sigmoid().cpu(), 
    inputs["original_sizes"].cpu(), 
    inputs["reshaped_input_sizes"].cpu(), binarize=False)


In [ ]:
inputs
outputs.pred_masks
outputs.keys()
print(outputs.pred_masks.shape)
print(inputs["original_sizes"])
print(inputs["reshaped_input_sizes"])


In [ ]:
probs[0].shape

In [ ]:
def show_mask(mask, ax, random_color):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([251/255, 252/255, 30/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor="blue", facecolor=(0, 0, 0, 0), lw=2))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(np.array(raw_image))
# ax[0].imshow(np.array(t_raw_image_np))
show_box(input_boxes, ax[0])
ax[0].set_title("Input Image and Bounding Box")
ax[0].axis("off")
ax[1].imshow(np.array(raw_image))
# ax[1].imshow(np.array(t_raw_image_np))
show_mask(mask=probs[0] > 0.5, ax=ax[1], random_color=False)
show_box(input_boxes, ax[1])
ax[1].set_title("MedSAM Segmentation")
ax[1].axis("off")
plt.show()

In [ ]:
print(outputs.pred_masks.sigmoid().cpu())
print(inputs["original_sizes"].cpu())
print(inputs["reshaped_input_sizes"].cpu())
probs = processor.image_processor.post_process_masks(
    outputs.pred_masks.sigmoid().cpu(), 
    inputs["original_sizes"].cpu(), 
    inputs["reshaped_input_sizes"].cpu(), binarize=False)

print(probs)

# Training

In [ ]:
# Define your dataset class
class ImageMaskDataset(Dataset):
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]
        return image, mask

In [ ]:
# Define your model
model = SamModel.from_pretrained("flaviagiammarino/medsam-vit-base")
# Modify the model's output layer for segmentation task
# For example, for binary segmentation, you can use a sigmoid activation
# model.config
# model.classifier = nn.Sequential(
#     nn.Linear(model.config.vision_config.projection_dim, 1),
#     nn.Sigmoid()
# )

In [ ]:
# save
torch.save(model.state_dict(), "/sddata/projects/MedSAM/MedSamBundle/models/model.pt")

In [ ]:
# load
model.load_state_dict(torch.load("/sddata/projects/MedSAM/MedSamBundle/models/model.pt"))


In [ ]:
model

In [ ]:
# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary segmentation
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
# Prepare your dataset and dataloaders
train_image = Image.open('/sddata/data/MedSAM/public_test_data_01_10_2023/Color_Fundus/visivite_GA.png')
train_mask = Image.open('/sddata/data/MedSAM/public_test_data_01_10_2023/labels/visivite_GA.png')

val_image = Image.open('/sddata/data/MedSAM/public_test_data_01_10_2023/Color_Fundus/76_year_old_GA.png')
val_mask = Image.open('/sddata/data/MedSAM/public_test_data_01_10_2023/labels/76_year_old_GA.png')


In [ ]:
train_image_np = np.array(train_image)
train_mask_np = np.array(train_mask)

val_image_np = np.array(val_image)
val_mask_np = np.array(val_mask)

In [ ]:
print(train_image_np.shape)
print(train_mask_np.shape)
print(val_image_np.shape)
print(val_mask_np.shape)


In [ ]:
train_image_np[:,:,0] # R
# train_image_np[:,:,1] # G
# train_image_np[:,:,2] # B
# train_image_np[:,:,3] # alpha

# train_image_np[500,500,0] # R pixel at 500, 500
# train_image_np[500,500,1]# G pixel at 500, 500
# train_image_np[500,500,2]# B pixel at 500, 500
# train_image_np[500,500,3]# A pixel at 500, 500

In [ ]:
# Define transformations (resize, convert to tensor)
transform = transforms.Compose([
    transforms.Resize((1024, 1024)),  # Resize the image and mask to the desired input size
    transforms.ToTensor(),           # Convert PIL Image to tensor
])

In [ ]:
# Apply transformations to image and mask
train_image = transform(train_image)
train_mask = transform(train_mask)
train_mask = (train_mask > 0.5).float()
# torch.Size([4, 1024, 1024])

val_image = transform(val_image)
val_mask = transform(val_mask)
val_mask = (val_mask > 0.5).float()
# torch.Size([4, 1024, 1024])


In [ ]:
# Add batch dimension to image and mask
train_image = train_image.unsqueeze(0)  # Add batch dimension
train_mask = train_mask.unsqueeze(0)    # Add batch dimension
# torch.Size([1, 4, 1024, 1024])

val_image = val_image.unsqueeze(0)  # Add batch dimension
val_mask = val_mask.unsqueeze(0)    # Add batch dimension
# torch.Size([1, 4, 1024, 1024])

In [ ]:
image_t = train_image[:,0:3,:,:]
mask_t = train_mask[:,3,:,:].unsqueeze(1)

image_v = val_image[:,0:3,:,:]
mask_v = val_mask[:,3,:,:].unsqueeze(1)

In [ ]:
# Check the shapes
print("Train Image shape:", image_t.shape)  # Should be (1, 3, 224, 224) if RGB image
print("Train Mask shape:", mask_t.shape)    # Should be (1, 1, 224, 224) if single-channel mask

print("Val Image shape:", image_v.shape)  # Should be (1, 3, 224, 224) if RGB image
print("Val Mask shape:", mask_v.shape)    # Should be (1, 1, 224, 224) if single-channel mask

In [ ]:
train_dataset = ImageMaskDataset(image_t, mask_t)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

val_dataset = ImageMaskDataset(image_v, mask_v)
val_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [ ]:
image_t.shape
train_dataset

In [ ]:
# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for images, masks in train_loader:
        pdb.set_trace()
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            val_loss += criterion(outputs, masks).item()

        val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item()}, Val Loss: {val_loss}")

# Save the trained model
torch.save(model.state_dict(), "trained_model.pth")